# Импорты

In [22]:
import requests
from fake_useragent import UserAgent
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as ec
from selenium.webdriver.common.action_chains import ActionChains
from selenium.common.exceptions import NoSuchElementException, TimeoutException
import pandas as pd
import os
import time

# Описание

**Задача:** Парсить товар, основные данные его и его отзывы.

**План:**
- Парсим информацию о каждом товаре в один .csv
- Парсим отзывы для каждого товара в отдельный .csv

**Откуда брать данные?** С популярных торговых площадок. Для начала спарсим с Wildberries и Ozon.

Парсить можно очень много, поэтому для начала разработки спарсим 3 категории по 100 товаров.

# Парсинг Wildberries

Установим ссылки на каталоги, которые хотим парсить

In [27]:
links_catalogs = [
    'https://www.wildberries.ru/catalog/yuvelirnye-ukrasheniya/koltsa',
    'https://www.wildberries.ru/catalog/aksessuary/galstuki-i-babochki',
    'https://www.wildberries.ru/catalog/knigi/nehudozhestvennaya-literatura/filosofiya'
]

In [28]:
COUNT_PRODUCTS_PARSE = 100  # Количество товаров, которые будем парсить со страницы каталога
COUNT_REVIEWS_PARSE = 100  # Количество отзывов, которые будем парсить со страницы товара

## Попытка парсинга с requests

In [6]:
ua = UserAgent()
user_agent = ua.random
headers = {
    'User-Agent': user_agent
}

Перед парсингом Wildberries ознакомился с их API. Понял, что он для моей задачи не подходит, так как он ориентирован для работы продавцов и Wildberries. Такая же история с Озоном.

In [7]:
for link in links_catalogs:
    responce = requests.get(url=link, headers=headers, timeout=4)
    print(responce.text)
    break

<!DOCTYPE html>
<html>

<head>
	<meta http-equiv="Content-Type" content="text/html; charset=UTF-8">
	<meta name="viewport" content="width=device-width, initial-scale=1, shrink-to-fit=no">
	<title></title>
</head>

<body>
    <style type="text/css">
		html,
		body {
			height: 100%;
		}
	
		body {
			box-sizing: border-box;
			padding: 0;
			margin: 0;
			font-family: system, -apple-system, BlinkMacSystemFont, 'Segoe UI', 'Segoe WP', Roboto, Ubuntu, Oxygen, Cantarell, 'Fira Sans', 'Helvetica Neue', Helvetica, 'Lucida Grande', 'Droid Sans', Tahoma, 'Microsoft Sans Serif', sans-serif;
			color: #000000;
		}
	
		.captcha {
			padding: 80px;
			display: flex;
			flex-direction: column;
			align-items: center;
			text-align: center;
		}
	
		.captcha>p {
			max-width: 510px;
			font-size: 16px;
			line-height: 22px;
			margin: 0;
		}
	
		.captcha__desc p {
			margin: 0 0 12px;
			font-size: 16px;
			line-height: 22px;
			color: #8B8B8B;
			max-width: 650px;
			word-break: break-word;
		}
	
		

Обнаружена динамическая подрузка, значит обычными requests спарсить не получится.

## Парсинг с selenium

Значит будет использовать selenium для парсинга данных с сайта с динамическими данными

Настроим selenium для будущей работы

In [19]:
service = Service(executable_path=ChromeDriverManager().install())

def open_chrome():
    driver = webdriver.Chrome(service=service)
    wait = WebDriverWait(driver, 15, poll_frequency=1)
    actions = ActionChains(driver)
    return driver, wait, actions

Спарсим данные с помощью selenium

In [ ]:
for link_catalog in links_catalogs:
    driver, wait, actions = open_chrome()
    driver.set_window_size(1280, 1000)
    driver.get(link_catalog)
    name_file = link_catalog.replace('https://www.wildberries.ru/catalog/', '').replace('/', '').replace('-', "_")

    links_products = []  # тут храним ссылки на страницы товаров
    page = 1
    i_real = 1
    # Сбор ссылок на товары из категории
    for i in range(1, COUNT_PRODUCTS_PARSE+1):
        locator = ('xpath', f'(//div[@class="product-card__wrapper"]/a)[{i_real}]')  # определить ссылку на продукт
        try:
            product = wait.until(ec.visibility_of_element_located(locator))  # найти продукт
        except TimeoutException:
            i_real = 1
            page += 1
            driver.get(f'{link_catalog}?sort=popular&page={page}')
            locator = ('xpath', f'(//div[@class="product-card__wrapper"]/a)[{i_real}]')  # определить ссылку на продукт
            product = wait.until(ec.visibility_of_element_located(locator))  # найти продукт
        links_products.append(product.get_attribute('href'))  # сохранить ссылку на продукт
        actions.move_to_element(product).perform()  # прокрутить до продукта
        driver.execute_script("window.scrollBy(0, 350);")
        i_real += 1

    df_products = pd.DataFrame(columns=["product_article", "product_name", "product_rating", "count_reviews"])
    # Обработка каждого заказа
    for link_product in links_products:
        driver.get(link_product)  # открыть страницу товара
        product_name = wait.until(ec.visibility_of_element_located(('xpath', '//h1[@class="product-page__title"]'))).text  # получить название товара
        product_id = wait.until(ec.visibility_of_element_located(('xpath', '//span[@id="productNmId"]'))).text  # получить название товара
        product_price = wait.until(ec.visibility_of_element_located(('xpath', '//span[@class="price-block__price"]'))).text  # получить цену товара
        product_rating = wait.until(ec.visibility_of_element_located(('xpath', '//span[contains(@class, "product-review__rating")]'))).text  # получить цену товара

        driver.get(f'https://www.wildberries.ru/catalog/{product_id}/feedbacks')
        df_reviews = pd.DataFrame(columns=["review_rating", "review_text", "review_text_plus", "review_text_minus"])
        for j in range(1, COUNT_REVIEWS_PARSE+1):
            prefix = f'(//ul[@class="comments__list"]/li)[{j}]'
            locator = ('xpath', f'(//ul[@class="comments__list"]/li)[{j}]')  # определить ссылку на отзыв
            try:
                review = wait.until(ec.visibility_of_element_located(locator))  # получить элемент с отзывом
            except TimeoutException:
                j -= 1
                break
            review_text = wait.until(ec.visibility_of_element_located(('xpath', f'{prefix}//div[@class="feedback__content"]'))).text  # получить текст отзыва
            review_rating = wait.until(ec.visibility_of_element_located(('xpath', f'{prefix}//span[contains(@class, "stars-line")]'))).get_attribute('class')[-1:]  # получить отзыв
            try:  # Получить текст достоинства
                review_text_plus = driver.find_element('xpath', f'{prefix}//div[contains(@class, "feedback__text--item-pro")]').text  # получить текст отзыва
            except NoSuchElementException:
                review_text_plus = None
            try:  # Получить текст недостатков
                review_text_negative = driver.find_element('xpath', f'{prefix}//div[contains(@class, "feedback__text--item-con")]').text  # получить текст отзыва
            except NoSuchElementException:
                review_text_negative = None
            actions.move_to_element(review).perform()  # прокрутить до продукта
            driver.execute_script("window.scrollBy(0, 350);")
            df_review = pd.DataFrame([{
                "review_rating": review_rating, 
                "review_text": review_text, 
                "review_text_plus": review_text_plus, 
                "review_text_minus": review_text_negative
            }])
            df_reviews = pd.concat([df_reviews, df_review], ignore_index=True)

        path_reviews = f'./../data/tests/raw/reviews/{name_file}'
        os.makedirs(path_reviews, exist_ok=True)
        df_reviews.to_csv(f'{path_reviews}/{product_id}.csv', index=False)
        df_product = pd.DataFrame([{
            "product_article": product_id, 
            "product_name": product_name, 
            "product_rating": product_rating, 
            "count_reviews": j
        }])
        df_products = pd.concat([df_products, df_product], ignore_index=True)

    path_products = './../data/tests/raw/catalogs'
    os.makedirs(path_reviews, exist_ok=True)
    df_products.to_csv(f'{path_products}/{name_file}.csv', index=False)

WebDriverException: Message: disconnected: Unable to receive message from renderer
  (failed to check if window was closed: disconnected: not connected to DevTools)
  (Session info: chrome=131.0.6778.109)
Stacktrace:
	GetHandleVerifier [0x003C3433+25059]
	(No symbol) [0x0034CE34]
	(No symbol) [0x0022BEC3]
	(No symbol) [0x0021D773]
	(No symbol) [0x0021D481]
	(No symbol) [0x0021B91B]
	(No symbol) [0x0021BE6B]
	(No symbol) [0x0021AB45]
	(No symbol) [0x002358D4]
	(No symbol) [0x002B4369]
	(No symbol) [0x00291EFC]
	(No symbol) [0x002AB51E]
	(No symbol) [0x00291C96]
	(No symbol) [0x00263FAC]
	(No symbol) [0x00264F3D]
	GetHandleVerifier [0x006B5593+3113795]
	GetHandleVerifier [0x006CA25A+3198986]
	GetHandleVerifier [0x006C2A32+3168226]
	GetHandleVerifier [0x004632A0+680016]
	(No symbol) [0x0035577D]
	(No symbol) [0x00352A28]
	(No symbol) [0x00352BC5]
	(No symbol) [0x00345820]
	BaseThreadInitThunk [0x75CA7BA9+25]
	RtlInitializeExceptionChain [0x7758C0CB+107]
	RtlClearBits [0x7758C04F+191]
